# Load Demo with DAG

In this we demonstrate loading of a sample data file [reduced_sample_data.json](../../data/reduced_sample_data.json) and showcase all the 
various resources that gets populated. We now are demonstrating using DAG

#### Pre-requisite
It is assumed that the setup steps are completed successfully. These are:
 - creating the database, schemas, stages
 - defining the external stage
 - defining the functions and stored procedures
 - defining the external tables and views

If you had not done this, use the streamlit (./bin/run_app.sh) to create these as defined in the subpage 'Setup'

In [18]:
# Initialization block
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json ,logging

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../../'
config = L.get_config(PROJECT_HOME_DIR)
sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.use_role(f'''{config['APP_DB']['role']}''')
sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
sp_session.use_warehouse(f'''{config['SNOW_CONN']['warehouse']}''')

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

### Initialization

,CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,VSEKAR,PUBLIC,INDSOL_CMSGOV_PRICING,PUBLIC


In [19]:
import pandas as pd
import os
from datetime import datetime
from datetime import timedelta
import time
import re

pd.set_option('display.max_colwidth', None)

def get_basename_of_datafile(p_datafile:str) -> str:
    base = os.path.basename(p_datafile)
    fl_base = os.path.splitext(base)
    return fl_base[0]

def get_cleansed_file_basename(p_datafile):
    fl_basename = get_basename_of_datafile(p_datafile)
    # Replace all non alphanumeric characters with _
    fl_name = re.sub('[^0-9a-zA-Z]+', '_', fl_basename)
    return fl_name

# INPUT_DATA_STAGE = config['APP_DB']['ext_stage']
INPUT_DATA_STAGE = 'data_stg'
DATA_STAGE_FOLDER = config['APP_DB']['folder_data']

DATA_FILE = '2022_10_01_priority_health_HMO_in-network-rates.zip'
DATA_FILE = 'reduced_sample_data.json'
DATA_FILE_BASENAME = get_basename_of_datafile(DATA_FILE)
DATA_FILE_BASENAME_CLEANSED = get_cleansed_file_basename(DATA_FILE)

TARGET_DATA_STAGE = config['APP_DB']['ext_stage']
TARGET_FOLDER = config['APP_DB']['folder_parsed']

SEGMENTS_PER_TASK = 1000

In [20]:
display(Markdown("### Cleanup block"))
# We will cleanup specific resources and artifacts from possible previous runs.

stmts = [
    f''' delete from segment_task_execution_status where data_file = '{DATA_FILE}'; '''
    ,f''' delete from task_to_segmentids where data_file = '{DATA_FILE}'; '''
    ,f''' delete from in_network_rates_file_header where data_file = '{DATA_FILE}'; '''
    ,f''' delete from in_network_rates_segment_header where data_file = '{DATA_FILE}'; '''
    ,f''' alter stage {INPUT_DATA_STAGE} refresh; '''
]    
    
print(' truncating tables ...')
for stmt in stmts:
    sp_session.sql(stmt).collect()

print(f''' cleaning up files in external stage under path {TARGET_FOLDER}/{DATA_FILE_BASENAME}/ ...''')

stmt = f''' select relative_path from directory(@{TARGET_DATA_STAGE}) where relative_path like '%{DATA_STAGE_FOLDER}/{DATA_FILE_BASENAME}/%'; '''
files = sp_session.sql(stmt).collect()
for r in files:
    stmt = f''' remove @{TARGET_DATA_STAGE}/{r['RELATIVE_PATH']}; '''
    sp_session.sql(stmt).collect()

### Cleanup block

 truncating tables ...
 cleaning up files in external stage under path raw_parsed/reduced_sample_data/ ...


---
## Data loading
We will be loading the segments and file header using DAG. 

In [21]:
# we build out the DAG
df = sp_session.call('in_network_rates_dagbuilder' ,f'{INPUT_DATA_STAGE}/{DATA_STAGE_FOLDER}' ,DATA_FILE 
    ,f"@{TARGET_DATA_STAGE}/{TARGET_FOLDER}" ,SEGMENTS_PER_TASK ,config['SNOW_CONN']['warehouse'])

sp_session.sql(f''' alter stage {TARGET_DATA_STAGE} refresh; ''').collect()
print(' Status of execution')
print(df)

 Status of execution
{
  "data_file": "reduced_sample_data.json",
  "root_task": "DAG_ROOT_reduced_sample_data",
  "status": true,
  "task_matrix_shape": [
    5,
    15
  ],
  "term_task": "TERM_tsk_reduced_sample_data"
}


The above operation results in defining the DAG in Snowflake like here. The task names are specific to the data file being parsed.
![](../../doc/soln_images/task_dags.png)

In [22]:
# Next we invoke the DAG

start_time = time.time()
print(f'Started at: {datetime.now().strftime("%H:%M:%S")}')

sql_stmts = [
    f''' alter warehouse {config['SNOW_CONN']['warehouse']} set warehouse_size = MEDIUM; '''
    # ,f''' truncate table segment_task_execution_status; '''
    # ,f''' truncate table task_to_segmentids; '''
    # ,f''' truncate table in_network_rates_file_header; '''
    # ,f''' truncate table in_network_rates_segment_header; '''
    ,f''' execute task DAG_ROOT_{DATA_FILE_BASENAME_CLEANSED}; '''
]
for stmt in sql_stmts:
    print(stmt)
    sp_session.sql(stmt).collect()

end_time = time.time()
print(f'Ended at: {datetime.now().strftime("%H:%M:%S")}')

elapsed_time = end_time - start_time
elapsed = str(timedelta(seconds=elapsed_time))
print(f'Elapsed: {elapsed}')

Started at: 14:20:38
 alter warehouse DEMO_BUILD_WH set warehouse_size = MEDIUM; 
 execute task DAG_ROOT_reduced_sample_data; 
Ended at: 14:20:40
Elapsed: 0:00:01.392551


---
## Inspection (TODO)

In [23]:
import time

continue_sleeping = True

while continue_sleeping == True: 
    time.sleep(2*60)

    sql_stmt = f'''
        select *
        from current_segment_parsing_tasks_v
        where l.data_file = '{DATA_FILE}'
        ;
    '''
    df_running = sp_session.table('current_segment_parsing_tasks_v').to_pandas()
    
    len_running_count = len(df_running)
    print(f'Current running tasks [{len_running_count}]...')
    display(df_running)
    
    sql_stmt = f'''
        select 
            count(l.*) as l_count
            ,count(r.*) as r_count
            ,l_count - r_count as row_count_diff
            -- l.*
        from segment_task_execution_status as l
            full outer join segments_counts_for_datafile_v as r
                on r.task_name = l.task_name
        where not (l.task_name  like any ('DAG_%' ,'TERM_%' ,'%T_FH_%' ))
            and JSON_EXTRACT_PATH_TEXT(l.task_ret_status ,'task_ignored_parsing') is null
            and l.data_file = '{DATA_FILE}'
    '''
    df = sp_session.sql(sql_stmt).to_pandas()
    row_count = df['ROW_COUNT_DIFF'][0]
    finished_row_count = df['R_COUNT'][0]
    print(f'Finished tasks [{row_count}] ...')
    if row_count == 0:
        print('Finished sleeping ...')
        continue_sleeping = False
        break

    display(df)
    print('\n ------------------------------ ')

Current running tasks [0]...


,TASK_NAME,START_TIME,BUCKET,ASSIGNED_TASK_NAME,FROM_IDX,TO_IDX,SEGMENTS_RECORD_COUNT


Finished tasks [0] ...
Finished sleeping ...


In [24]:
display(Markdown("The table in_network_rates_file_header holds the file header elements"))

df = (sp_session.table('in_network_rates_file_header')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .to_pandas())

display(df)

The table in_network_rates_file_header holds the file header elements

,DATA_FILE,HEADER,INSERTED_AT
0,reduced_sample_data.json,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""last_updated_on"": ""2022-10-01"",\n ""reporting_entity_name"": ""Priority Health"",\n ""reporting_entity_type"": ""health insurance issuer"",\n ""total_segments"": 10,\n ""version"": ""http://json-schema.org/draft-07/schema#""\n}",2023-01-06 11:20:59.597


In [25]:
print(' The HEADER has the following data ')
json.loads(df['HEADER'][0])

 The HEADER has the following data 


{'DATA_FILE': 'reduced_sample_data.json',
 'last_updated_on': '2022-10-01',
 'reporting_entity_name': 'Priority Health',
 'reporting_entity_type': 'health insurance issuer',
 'total_segments': 10,
 'version': 'http://json-schema.org/draft-07/schema#'}

In [26]:
display(Markdown("The table in_network_rates_segment_header lists the various segments that were loaded"))

df = (sp_session.table('in_network_rates_segment_header')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)

The table in_network_rates_segment_header lists the various segments that were loaded

,DATA_FILE,SEGMENT_ID,NEGOTIATED_RATES_INFO,NEGOTIATED_RATES_COUNT,BUNDLED_CODES_COUNT,COVERED_SERVICES_COUNT,INSERTED_AT
0,reduced_sample_data.json,ffs::cpt::86160::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::86160::"",\n ""SEQ_NO"": 4,\n ""billing_code"": ""86160"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Complement Antigen Each Component"",\n ""name"": ""Complement Antigen Each Component"",\n ""negotiation_arrangement"": ""ffs""\n}",2779,-1,-1,2023-01-06 11:21:23.523
1,reduced_sample_data.json,ffs::cpt::92583::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::92583::"",\n ""SEQ_NO"": 0,\n ""billing_code"": ""92583"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Select Picture Audiometry"",\n ""name"": ""Select Picture Audiometry"",\n ""negotiation_arrangement"": ""ffs""\n}",5323,-1,-1,2023-01-06 11:21:08.985
2,reduced_sample_data.json,ffs::cpt::62322::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::62322::"",\n ""SEQ_NO"": 1,\n ""billing_code"": ""62322"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Injection(s), of diagnostic or therapeutic substance(s) (eg, anesthetic, antispasmodic, opioid, steroid, other solution), not including neurolytic substances, including needle"",\n ""name"": ""Injection(s), of diagnostic or therapeutic substance(s) (eg, anesthetic, antispasmodic, opioid, steroid, other solution), not including neurolytic substances, including needle"",\n ""negotiation_arrangement"": ""ffs""\n}",4170,-1,-1,2023-01-06 11:21:13.598
3,reduced_sample_data.json,ffs::cpt::33235::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::33235::"",\n ""SEQ_NO"": 8,\n ""billing_code"": ""33235"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Removal of Perm Pacemaker; and Trnsvenous Electrodes, Dual Lead System"",\n ""name"": ""Removal of Perm Pacemaker; and Trnsvenous Electrodes, Dual Lead System"",\n ""negotiation_arrangement"": ""ffs""\n}",5029,-1,-1,2023-01-06 11:21:37.573
4,reduced_sample_data.json,ffs::cpt::15004::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::15004::"",\n ""SEQ_NO"": 7,\n ""billing_code"": ""15004"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""PREP SITE F/S/N/H/F/G/M/D GT 1ST 100 SQ CM/1PCT"",\n ""name"": ""PREP SITE F/S/N/H/F/G/M/D GT 1ST 100 SQ CM/1PCT"",\n ""negotiation_arrangement"": ""ffs""\n}",9227,-1,-1,2023-01-06 11:21:33.899


In [27]:
print(' The NEGOTIATED_RATES_INFO has the following data ')
json.loads(df['NEGOTIATED_RATES_INFO'][0])

 The NEGOTIATED_RATES_INFO has the following data 


{'DATA_FILE': 'reduced_sample_data.json',
 'SEGMENT_ID': 'ffs::cpt::86160::',
 'SEQ_NO': 4,
 'billing_code': '86160',
 'billing_code_type': 'CPT',
 'billing_code_type_version': '',
 'description': 'Complement Antigen Each Component',
 'name': 'Complement Antigen Each Component',
 'negotiation_arrangement': 'ffs'}

In [28]:
para = f''' 
The parsed data are stored as parquet files in the external stage at: @{TARGET_DATA_STAGE}/{TARGET_FOLDER}. The directory structure follows the format:

<ext stage>/<folder_parsed>/<data file basename (ex: reduced_sample_data)/<segment_id>/<segment type (negotiated_rates)>/data_<seq_no>_<chunk_no>.parquet.gz

Description:
- ext stage : external stage name
- folder_parsed : configured value of APP_DB.folder_parsed in config.ini
- data file basename : the data file basename, without the file extension
- segment_id : a unique identifier for the segment, this is a composite key of <negotiation_arrangement>::<billing_code_type>::<billing_code>::<billing_code_type_version>
- segment type : indicates the segment children type, this could be either of negotiated_rates/bundled_codes/covered_services
- The file which will contain the record, stored in parquet file.
'''
print(para)

stmt = f''' select relative_path from directory(@{TARGET_DATA_STAGE}) where relative_path like '%{TARGET_FOLDER}/{DATA_FILE_BASENAME}%' limit 5; '''
df = sp_session.sql(stmt).to_pandas()
display(df)

 
The parsed data are stored as parquet files in the external stage at: @ext_data_stg/raw_parsed. The directory structure follows the format:

<ext stage>/<folder_parsed>/<data file basename (ex: reduced_sample_data)/<segment_id>/<segment type (negotiated_rates)>/data_<seq_no>_<chunk_no>.parquet.gz

Description:
- ext stage : external stage name
- folder_parsed : configured value of APP_DB.folder_parsed in config.ini
- data file basename : the data file basename, without the file extension
- segment_id : a unique identifier for the segment, this is a composite key of <negotiation_arrangement>::<billing_code_type>::<billing_code>::<billing_code_type_version>
- segment type : indicates the segment children type, this could be either of negotiated_rates/bundled_codes/covered_services
- The file which will contain the record, stored in parquet file.



,RELATIVE_PATH
0,raw_parsed/reduced_sample_data/ffs::cpt::15004::/negotiated_rates/data_7_0.parquet.gz
1,raw_parsed/reduced_sample_data/ffs::cpt::15004::/negotiated_rates/data_7_1.parquet.gz
2,raw_parsed/reduced_sample_data/ffs::cpt::33235::/negotiated_rates/data_8_0.parquet.gz
3,raw_parsed/reduced_sample_data/ffs::cpt::33235::/negotiated_rates/data_8_1.parquet.gz
4,raw_parsed/reduced_sample_data/ffs::cpt::62322::/negotiated_rates/data_1_0.parquet.gz


In [29]:
para = f''' 
The parquet file can be queried inside Snowflake via external table: ext_negotiated_arrangments_staged. Each of the folder structure is partitioned to seperate columns , which can help with pruning to specific segments.
The value contains the negotiated_rates and other children elements stored in JSON format
'''
print(para)

pd.set_option('display.max_colwidth', 25)
df = (sp_session.table('ext_negotiated_arrangments_staged')
        .filter(F.col('P_DATA_FL') == F.lit(DATA_FILE_BASENAME))
        .sample(n=5)
        .to_pandas())

display(df)


 
The parquet file can be queried inside Snowflake via external table: ext_negotiated_arrangments_staged. Each of the folder structure is partitioned to seperate columns , which can help with pruning to specific segments.
The value contains the negotiated_rates and other children elements stored in JSON format



,VALUE,P_DATA_FL,P_SEGMENT_ID,P_NEGOTIATION_ARRANGEMENT,P_BILLING_CODE_TYPE,P_BILLING_CODE,P_BILLING_CODE_TYPE_VERSION,P_SEGMENT_TYPE
0,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::85041::,ffs,cpt,85041,,negotiated_rates
1,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::j9144::,ffs,cpt,j9144,,negotiated_rates
2,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::76391::,ffs,cpt,76391,,negotiated_rates
3,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::92583::,ffs,cpt,92583,,negotiated_rates
4,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::15004::,ffs,cpt,15004,,negotiated_rates


In [30]:
print(' a sample view of one of the records')
j = json.loads(df['VALUE'][0])
print(j)

 a sample view of one of the records
{'CHUNK_NO': 0, 'DATA_FILE': 'reduced_sample_data.json', 'NEGOTIATED_RATES': [{'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 1.51, 'negotiated_type': 'negotiated', 'service_code': ['11']}], 'provider_groups': [{'npi': [1902809940], 'tin': {'type': 'ein', 'value': '133757370'}}]}, {'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 1.51, 'negotiated_type': 'negotiated', 'service_code': ['11']}], 'provider_groups': [{'npi': [1104829159], 'tin': {'type': 'ein', 'value': '840611484'}}]}, {'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 2.65, 'negotiated_type': 'fee schedule', 'service_code': ['11', '22']}], 'provider_groups': [{'npi': [1245307818], 'tin': {'type': 'ein', 'value': '520890739'}}]}, {'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-

In [31]:
display(Markdown("The table segment_task_execution_status contains the audit of execution for the above stored procedures"))

pd.set_option('display.max_colwidth', None)
df = (sp_session.table('segment_task_execution_status')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)

The table segment_task_execution_status contains the audit of execution for the above stored procedures

,DATA_FILE,TASK_NAME,START_TIME,END_TIME,TASK_RET_STATUS,INSERTED_AT
0,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_4001_5000,2023-01-06 11:20:58.665,2023-01-06 11:21:01.983,"{\n ""EOF_Reached"": true,\n ""Parsing_error"": """",\n ""data_file"": ""reduced_sample_data.json"",\n ""elapsed"": ""=> 0:00:02.535023 "",\n ""end_rec_num"": 5000,\n ""last_seg_no"": 10,\n ""start_rec_num"": 4001,\n ""task_ignored_parsing"": false\n}",2023-01-06 11:20:58.665
1,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_27001_28000,2023-01-06 11:22:10.743,2023-01-06 11:22:11.983,"{\n ""data_file"": ""reduced_sample_data.json"",\n ""elapsed"": ""=> 0:00:00.359950 "",\n ""end_rec_num"": 28000,\n ""start_rec_num"": 27001,\n ""task_ignored_parsing"": true,\n ""task_parsing_ignore_message"": ""The segment range is greater than the total segments -2 in the file, hence ignore further parsing""\n}",2023-01-06 11:22:10.743
2,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_21001_22000,2023-01-06 11:22:06.827,2023-01-06 11:22:07.724,"{\n ""data_file"": ""reduced_sample_data.json"",\n ""elapsed"": ""=> 0:00:00.342671 "",\n ""end_rec_num"": 22000,\n ""start_rec_num"": 21001,\n ""task_ignored_parsing"": true,\n ""task_parsing_ignore_message"": ""The segment range is greater than the total segments -2 in the file, hence ignore further parsing""\n}",2023-01-06 11:22:06.827
3,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_26001_27000,2023-01-06 11:22:18.840,2023-01-06 11:22:20.177,"{\n ""data_file"": ""reduced_sample_data.json"",\n ""elapsed"": ""=> 0:00:00.358640 "",\n ""end_rec_num"": 27000,\n ""start_rec_num"": 26001,\n ""task_ignored_parsing"": true,\n ""task_parsing_ignore_message"": ""The segment range is greater than the total segments -2 in the file, hence ignore further parsing""\n}",2023-01-06 11:22:18.840
4,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_3001_4000,2023-01-06 11:21:05.022,2023-01-06 11:21:06.708,"{\n ""data_file"": ""reduced_sample_data.json"",\n ""elapsed"": ""=> 0:00:00.327903 "",\n ""end_rec_num"": 4000,\n ""start_rec_num"": 3001,\n ""task_ignored_parsing"": true,\n ""task_parsing_ignore_message"": ""The segment range is greater than the total segments -2 in the file, hence ignore further parsing""\n}",2023-01-06 11:21:05.022


In [32]:
display(Markdown("The view segments_counts_for_datafile_v, based of table segment_task_execution_status, will help to identify the number of negotiated_arrangement segments for a specific data file. This view is populated once all the segments in a specific data files are parsed out"))

df = (sp_session.table('segments_counts_for_datafile_v')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)

The view segments_counts_for_datafile_v, based of table segment_task_execution_status, will help to identify the number of negotiated_arrangement segments for a specific data file. This view is populated once all the segments in a specific data files are parsed out

,DATA_FILE,TASK_NAME,START_REC_NUM,END_REC_NUM,TOTAL_NO_OF_SEGMENTS,ELAPSED,PARSING_ERROR,EOF_REACHED
0,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_0_1000,0,1000,10,0,,true
1,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_4001_5000,4001,5000,10,1,,true


--- 
### Closeout

    With that we are finished this section of the demo setup

In [33]:
# sp_session.close()
print('Finished!!!')

Finished!!!
